# Testing the model

In [6]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from transformers import BertTokenizer, BertForSequenceClassification


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the trained model

In [8]:
# Load the tokenizer and model from the saved directory
model_name = "/content/drive/MyDrive/project-toxic/toxic-model-v2"
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
Bert_Model = BertForSequenceClassification.from_pretrained(model_name)

In [9]:
def predict_user_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer):
    user_input = [input_text]

    user_encodings = tokenizer(user_input, truncation=True, padding=True, return_tensors="pt")

    user_dataset = TensorDataset(user_encodings['input_ids'], user_encodings['attention_mask'])

    user_loader = DataLoader(user_dataset, batch_size=1, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in user_loader:
            input_ids, attention_mask = [t for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)

    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)
    return predicted_labels[0].tolist()

In [20]:
def predict_label(predicted_label):
    labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    labels_pridicted = ""
    for i in range(len(predicted_label)):
        if predicted_label[i] == 1:
            labels_pridicted += labels[i] + " "

    if labels_pridicted == "":
        print("Your comment is not toxic.")
    else:
        print(f'Prdicted label(s) for your comment is {labels_pridicted}.')



In [21]:
labels_pridicted = ""
if labels_pridicted == "":
  print("hi")

hi


In [22]:
text = "I'll kill you"
predicted_labels = predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text)
predict_label(predicted_labels)

Prdicted label(s) for your comment is toxic threat .


In [25]:
text = "I am happy"
predicted_labels = predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text)
predict_label(predicted_labels)

Your comment is not toxic.


In [19]:
text = input("Enter your comment: ")
predicted_labels = predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text)
predict_label(predicted_labels)

Enter your comment: I am good
Prdicted label(s) for your comment is not toxic.
